In [ ]:
import requests
import pandas as pd
from opnieuw import retry
from requests.exceptions import HTTPError, ProxyError
from tinydb import TinyDB, Query

### Functions

In [ ]:
# Get Wallapop User Details
@retry(
  retry_on_exceptions=(ConnectionError, HTTPError, ProxyError),
  max_calls_total=4,
  retry_window_after_first_call_in_seconds=60,
)
def get_user(user_id):
  url = f"https://api.wallapop.com/api/v3/users/{user_id}"
  payload = {}
  headers = {
    'Accept': 'application/json, text/plain, */*',
    'Cache-Control': 'no-cache',
    'Connection': 'keep-alive',
    'Origin': 'https://es.wallapop.com',
    'Pragma': 'no-cache',
    'Referer': 'https://es.wallapop.com/',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-site',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36',
    'accept-language': 'es,en-GB;q=0.9,en-US;q=0.8,en;q=0.7',
    'deviceos': '0',
    'mpid': '8966409109343412630',
    'sec-ch-ua': '"Not/A)Brand";v="8", "Chromium";v="126", "Google Chrome";v="126"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'x-appversion': '82350',
    'x-deviceid': 'fa090a97-4fb2-4423-ae97-bdd5688c75b6',
    'x-deviceos': '0'
  }
  response = requests.request("GET", url, headers=headers, data=payload)
  if response.status_code == 200:
    return response.json()
  else:
    print(f'Status Code {response.status_code}')


In [ ]:
# Iterate the Users list
def iterate_users(users):
    db=TinyDB("D:/Proyecto ironhack/files/users.json")
    table=db.table("users")
    Serie=Query()
    i=0
    for user_id in users:
        try:
            i+=1
            print(f'Obteniendo usuario {i}/{len(users)} - ID: {user_id}')
            # time.sleep(random.random()*2)
            response_user = get_user(user_id)
            table.upsert(response_user, Serie.id==response_user.get('id', 'N/A'))
        except:
            continue

### Main Code

In [ ]:
if __name__ == "__main__":
    df = pd.read_csv('D:/Proyecto ironhack/files/spain_dataset.csv')
    users = list(set(df['user_id'].values))
    iterate_users(users)
    df = pd.read_json("D:/Proyecto ironhack/files/users.json")
    df = pd.json_normalize(df['users'])
    df.to_csv("D:/Proyecto ironhack/files/users_dataset.csv", index = False)